In [1]:
from chat_engine import ContextChatEngine
from kgpchatroom import KGPChatroomModel
LLM = KGPChatroomModel().get_model()
from llama_index.core.memory import ChatMemoryBuffer
pc_index = KGPChatroomModel().load_vector_index("Career")
from template import Template
chat_template = Template.get_template()
memory =  ChatMemoryBuffer.from_defaults(token_limit=4000)
chat_engine = ContextChatEngine.from_defaults(retriever=pc_index.as_retriever(),memory=memory,system_prompt=chat_template)

In [7]:
def chat_with_engine(chat_engine):
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break
        response = chat_engine.stream_chat(user_input)
        print("Chatbot: ", end="")
        for token in response.response_gen:
            print(token, end="")
        print()

In [8]:
chat_with_engine(chat_engine)

Chatbot: Hey!  What's up?  How can I help you with your career prep at KGP?

Chatbot: I'm not able to help you with that.  My purpose is to help IIT Kharagpur students with their career development, and providing assistance with cheating on exams goes against academic integrity and my programming.  Cheating can have serious consequences, including expulsion from the institute.  It's much better to focus on preparing honestly and building your skills.  Is there anything else I can help you with regarding your placement preparations?

Chatbot: The CDC office is located in Nalanda, the academic complex at IIT Kharagpur.  I don't have the exact room number, but you should be able to find it easily by looking at the maps available online or asking around Nalanda.

Chatbot: Goodbye!


In [15]:
history = chat_engine.chat_history

In [28]:
conversation_history = ""
for item in range(len(history)):
    message = f"User asked: {history[item].content} at {history[item].additional_kwargs["user_timestamp"]}" if item % 2 == 0 else f"Assistant replied: {history[item].content} at {history[item].additional_kwargs["assistant_timestamp"]}"
    conversation_history += f"{item + 1}. {message}\n"
    
tags_prompt_template = f"""Task: Generate a list of relevant tags based on Named Entity Recognition (NER) from the following conversation. 
The tags should include important people, organizations, locations, products, topics, and any other significant entities or concepts discussed.

Conversation History:
{conversation_history}

Instructions:
- Extract key entities mentioned in the conversation, including but not limited to:
  - Person names
  - Organizations
  - Locations
  - Products or brands
  - Events or key topics
- Focus on concepts that could summarize or represent the essence of the conversation.
- Return 20 tags as a simple list, separating each with a comma"""

tags = LLM.complete(tags_prompt_template)

tags.text.replace("\n", ", ")
#get the tags as a list
tags_list = tags.text.split(", ")

#iterate through each word and remove newline characters and spaces before and after the word
tags_list = [tag.strip() for tag in tags_list]